In [ ]:
import sys
import os
sys.path.append("../TychePlot/")
from PDF import PDF
from Plot import Plot
from multiprocessing import Pool

In [ ]:
#columns
axis1=1
axis2=2
axis3=3

In [ ]:
#obligatory
title="Title"
name="xxx"
fileList=[["file1"],["file2"]] #List of List
desiredPlots=[ # List of dictionaries
    {
        "xCol":axis1,
        "yCol":axis2
    }
]

In [ ]:
#optional
scaled=False #True if scaled graphs are desired 
customLims=False # better pick False running first Time or if no Limits are given
inputParameters={
                    "customFontsize":[12,12,8,10,10],
                    #"labels":["label1","label2"]
                    #"showColLabelUnit":["","Quantity (Unit)","Quantity (Unit)"],
                    "title":title,
                }
inputParametersForScaled=inputParameters.copy()
inputParametersForScaled.update({
                    "scaleX":0.47,
                    "customFontsize":[10,10,4,6,6],
                    "ax2Labels":False,
                    "titleBool":False
                    })

In [ ]:
def initPlot(xCol=1, showColTup=(2,3), customInputParameters=None):
    if customInputParameters is not None:
        inputParameters.update(customInputParameters)
        inputParametersForScaled.update(customInputParameters)
    scPlot=None
    if customLims:
        plot=Plot(name, 
                  fileList,     
                  xCol=xCol,     
                  showColTup=showColTup,     
                  xLimOrig=voltageLims[showColTup[0]],         
                  showColAxLim=yAxisLims, 
                  **inputParameters)
        if scaled:
            scPlot=Plot(name, 
                        fileList,      
                        xCol=xCol,     
                        showColTup=showColTup, 
                        xLimOrig=voltageLims[showColTup[0]], 
                        showColAxLim=yAxisLims, 
                        **inputParametersForScaled)
    else:
        plot=Plot(name, 
                  fileList,            
                  xCol=xCol,        
                  showColTup=showColTup,           
                  **inputParameters)
        if scaled:
            scPlot=Plot(name, fileList,
                        xCol=xCol, 
                        showColTup=showColTup,
                        **inputParametersForScaled)
    if scPlot is None:
        return [plot]
    return [plot,scPlot]

In [ ]:
def buildPlotList(desiredPlot):
    try:
        yCol2=desiredPlot["yCol2"]
    except KeyError:
        yCol2=0
    try:
        cusPara=desiredPlot["custom"]
    except KeyError:
        cusPara=None
    if desiredPlot["yCol"]==0 or desiredPlot["xCol"]==0:
        raise
    return initPlot(xCol=desiredPlot["xCol"], showColTup=(desiredPlot["yCol"],yCol2), customInputParameters=cusPara)

In [ ]:
def processPlotPair(plotpair):
    return [plot.doPlot() for plot in plotpair]

In [ ]:
#multithreaded
pool = Pool(os.cpu_count())
plotList=pool.map(buildPlotList,desiredPlots)
multiOutput=pool.map(processPlotPair,plotList)
pool.close()

In [ ]:
plots=[[plotOutput[0] for plotOutput in plotPair] for plotPair in multiOutput]
files=[[plotOutput[1] for plotOutput in plotPair] for plotPair in multiOutput]

In [ ]:
PDF(files[0][0], size=(700,700*3/4)) 